# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [9]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')

print(df.info())
print('\n',df.action.value_counts())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None

 view     6328
click    1860
Name: action, dtype: int64


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [17]:
views = set(df[df.action=='view']['id'].unique())
clicks = set(df[df.action=='click']['id'].unique())

print(f"Number of viewers: {len(views)} \tNumber of clickers: {len(clicks)}")
print(f"Number of viewers who didn't click: {len(views - clicks)}")
print(f"Number of clickers who didn't view: {len(clicks - views)}")

Number of viewers: 6328 	Number of clickers: 1860
Number of viewers who didn't click: 4468
Number of clickers who didn't view: 0


In [29]:
#Your code here
exids = set(df[df.group=='experiment']['id'].unique())
conids = set(df[df.group=='control']['id'].unique())
print(f"Overlap between experiment and control groups: {len(exids&conids)}")

Overlap between experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [32]:
df['count'] = 1

In [33]:
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print(f"Sample sizes:\tControl: {len(control)}\tExperiment: {len(experiment)}")
print(f"Total Clicks:\tControl: {control.click.sum()}\tExperiment: {experiment.click.sum()}")
print(f"Average click rate:\tControl: {control.click.mean()}\tExperiment: {experiment.click.mean()}")
control.head()


Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [39]:
import flatiron_stats as flat
wt = flat.p_value_welch_ttest(control.click, experiment.click)
print(f"Welch's t-test p-value: {wt}")

Welch's t-test p-value: 0.004466402814337078


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [41]:
#Your code here
control_mean = control.click.mean()
expected_experiment_clicks_under_null = control_mean * len(experiment)
print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [43]:
#Your code here
import numpy as np
n = len(experiment)
p = control_mean
var = n*p*(1-p)
std = np.sqrt(var)

In [44]:
actual_exp_clicks = experiment.click.sum()
z_score = (actual_exp_clicks - expected_experiment_clicks_under_null)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [45]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z_score) 
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Yes, despite the experiment p-value being slightly lower than the control, Ho can still be confidently rejected; in other words the experimental web page yields a greater CTR (click-through-rate).

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.